In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import csv

1. Collect all Course links from html page

Get Links to all classes

In [ ]:
filename = "SOC/Schedule of Classes.html"
with open(filename, "r") as f:
    content = f.read()

pattern = r"https://app.testudo.umd.edu/soc/202308/(.{4})"
matches = re.findall(pattern, content)

urls = []
for match in matches:
    url = "https://app.testudo.umd.edu/soc/202308/" + match
    urls.append(url)


def scrape_class_ids(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    class_ids = []

    for course in soup.find_all('div', {'class': 'course'}):
        class_id = course.find('div', {'class': 'course-id'}).text.strip()
        class_ids.append(class_id)

    return class_ids


url_class_id = []
updated_urls  = [s.replace("202308", "202301") for s in urls]

for url in updated_urls:
    class_ids = scrape_class_ids(url)
    for class_id in class_ids:
        new_url = url + '/' + class_id
        url_class_id.append(new_url)


updated_url_class_id  = [s.replace("202308", "202301") for s in url_class_id]

#print(updated_url_class_id)


2. Extract course information

In [ ]:


def scrape_section_data(url):
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    html_content = str(soup)

    section_data = []
    if "No courses matched your search filters above." in html_content or \
       "Contact department or instructor for details." in html_content or \
       '<span class="building-code">TBA</span>' in html_content or \
       '<span class="section-days">TBA</span>' in html_content or \
       '<span class="class-start-time">TBA</span>' in html_content or \
       '<span class="class-end-time">TBA</span>' in html_content or \
       '<span class="section_id">TBA</span>' in html_content or \
       '<span class="course_id">TBA</span>' in html_content:
        print(f"{url} No courses found for ")
        return []


    for section in soup.find_all('div', {'class': 'section delivery-f2f'}):
        section_id = section.find('span', {'class': 'section-id'}).text.strip()
        total_seats_count = section.find('span', {'class': 'total-seats-count'}).text.strip()
        open_seats_count = section.find('span', {'class': 'open-seats-count'}).text.strip()
        section_days = section.find('span', {'class': 'section-days'}).text.strip()
        class_start_time = section.find('span', {'class': 'class-start-time'}).text.strip()
        class_end_time = section.find('span', {'class': 'class-end-time'}).text.strip()
        class_building = section.find('span', {'class': 'class-building'}).text.strip()
        class_room = section.find('span', {'class': 'class-room'}).text.strip()

        section_data.append({
        'course_id': url[-7:],
        'section_id': section_id,
        'total_seats_count': total_seats_count,
        'open_seats_count': open_seats_count,
        'section_days': section_days,
        'class_start_time': class_start_time,
        'class_end_time': class_end_time,
        'class_building': class_building.replace('\n', ''),
        'class_room': class_room
        })
    return section_data
    
urls_testing = ['https://app.testudo.umd.edu/soc/202301/AASP/AASP255',
 'https://app.testudo.umd.edu/soc/202308/AASP/AASP100H',
 'https://app.testudo.umd.edu/soc/202308/AASP/AASP101']

fieldnames = ['course_id', 'section_id', 'total_seats_count', 'open_seats_count', 'section_days', 'class_start_time', 'class_end_time', 'class_building', 'class_room']

with open('section_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csv_writer.writeheader()

    for url in updated_url_class_id:
        section_data = scrape_section_data(url)
        csv_writer.writerows(section_data)
        #print(section_data)updated_url_class_id

